In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import datetime
import plotly.express as px

In [ ]:
# Set options to display all rows, all columns, and full column width
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000) # Adjust as needed for your console/display

In [ ]:
transaction1 = pl.read_parquet('/project/ai901504-ai0004/501641_Big/week3/demand-forecasting-non-duck-product/problem1_filter_transaction_price_1.parquet')
# transaction2 = pl.read_parquet('/project/ai901504-ai0004/501641_Big/week3/demand-forecasting-non-duck-product/problem1_filter_transaction_price_2.parquet')
event_date = pl.read_parquet('/project/ai901504-ai0004/501641_Big/week3/demand-forecasting-non-duck-product/event_date.parquet')
# price = pl.read_parquet('/content/kaggle_competition/problem1_price.parquet')

# Data


### price_df

In [ ]:
# price

### EDA

In [ ]:
transaction1

In [ ]:
pl.Config.set_tbl_cols(-1)
print(transaction1.null_count())

In [ ]:
print(transaction1.select(pl.col("WarehouseBKey").n_unique()).item())
print(transaction1.select(pl.col("CustomerBKey").n_unique()).item())
print(transaction1.select(pl.col("CustomerArea3NameLocal").n_unique()).item())
print(transaction1.select(pl.col("ProductForPlan1").n_unique()).item())
print(transaction1.select(pl.col("ProductForPlan8").n_unique()).item())
print(transaction1.select(pl.col("ProductForPlan10").n_unique()).item())
print(transaction1.select(pl.col("MODEL_4").n_unique()).item())
print(transaction1.select(pl.col("region").n_unique()).item())

### Group

In [ ]:
groupmodel_4 = transaction1.group_by(['MODEL_4', 'ProductForPlan10']).agg([
    pl.col('OrderWeight').sum(),
    pl.col('OrderWeight').mean().alias("OrderWeight_mean"),
    pl.col('OrderWeight').std().alias("OrderWeight_std"),
    pl.col('OrderWeight').median().alias("OrderWeight_median"),
    pl.col('OrderWeight').mode().alias("OrderWeight_mode"),
])


In [ ]:
groupmodel_4

### Visualization

In [ ]:
import random

def random_person_history_on_one_item(df):
    # Get unique list of customers
    list_people = df.select("CustomerBKey").unique().to_series().to_list()
    random_person = random.choice(list_people)

    # Get all transactions for that person
    random_person_history = df.filter(pl.col("CustomerBKey") == random_person)

    # Get unique list of products for that person
    list_items = random_person_history.select("ProductBKey").unique().to_series().to_list()
    random_item = random.choice(list_items)

    # Filter transactions for that person on one product
    person_history_on_item = random_person_history.filter(pl.col("ProductBKey") == random_item)

    # Visualize
    person_history_on_item = person_history_on_item.sort("OrderDate")
    df_plot = person_history_on_item.select(["OrderDate", "OrderWeight"]).to_pandas()

    plt.figure(figsize=(10, 6))
    plt.plot(df_plot["OrderDate"], df_plot["OrderWeight"], marker='o', linestyle='-')
    plt.title(f"OrderWeight Over Time\nCustomerBKey: {random_person}, ProductBKey: {random_item}")
    plt.xlabel("OrderDate")
    plt.ylabel("OrderWeight")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(True)
    plt.show()

    return person_history_on_item

In [ ]:
random_person = random_person_history_on_one_item(transaction1)
print(random_person)

In [ ]:
def visualize_group_byproduct(df, column=str):
    # Step 1: Get list of unique product categories
    list_product = df.select("ProductForPlan10").unique().to_series().to_list()

    # Step 2: Pick one randomly
    random_product = random.choice(list_product)

    # Step 3: Filter rows for that product category
    random_product_sale = df.filter(pl.col("ProductForPlan10") == random_product)

    # Step 4: Prepare data for plotting
    product_names = random_product_sale["MODEL_4"].to_list()
    weights = random_product_sale[column].to_list()

    # Step 5: Create Plotly bar chart
    fig = px.bar(
        x=weights,
        y=product_names,
        orientation="h",
        labels={"x": "Total OrderWeight", "y": "District (MODEL_4)"},
        title=f"Sales by District for Product Category: {random_product}",
    )

    # Optional: Highest value on top
    fig.update_layout(yaxis=dict(autorange="reversed"))

    # Show the interactive plot
    fig.show()

    return random_product_sale

In [ ]:
def visualize_group_byregion(df, column=str):
    # Step 1: Get list of unique product categories
    list_product = df.select("MODEL_4").unique().to_series().to_list()

    # Step 2: Pick one randomly
    random_product = random.choice(list_product)

    # Step 3: Filter rows for that product category
    random_product_sale = df.filter(pl.col("MODEL_4") == random_product)

    # Step 4: Prepare data for plotting
    product_names = random_product_sale["ProductForPlan10"].to_list()
    weights = random_product_sale[column].to_list()

    # Step 5: Create Plotly bar chart
    fig = px.bar(
        x=weights,
        y=product_names,
        orientation="h",
        labels={"x": "Total OrderWeight", "y": "Item (Plan 10)"},
        title=f"Sales by Product for district : {random_product}",
    )

    # Optional: Highest value on top
    fig.update_layout(yaxis=dict(autorange="reversed"))

    # Show the interactive plot
    fig.show()

    return random_product_sale

In [ ]:
random_item = visualize_group_byproduct(groupmodel_4, 'OrderWeight_mean')
print(random_item)

In [ ]:
random_district = visualize_group_byregion(groupmodel_4, 'OrderWeight_mean')
print(random_district)

### ภาคใต้

In [ ]:
print(transaction['region'].unique())

In [ ]:
transaction = transaction.filter(pl.col("region") == "ภาคใต้")

In [ ]:
transaction

In [ ]:
import polars as pl
import random
import plotly.graph_objects as go

def visual_stat_on_plan10(df: pl.DataFrame) -> None:
    # Step 1: Randomly select a region (MODEL_4)
    list_region = df.select("MODEL_4").unique().to_series().to_list()
    random_region = random.choice(list_region)

    # Step 2: Filter rows for that region
    region_df = df.filter(pl.col("MODEL_4") == random_region)

    # Step 3: Randomly select a product (ProductForPlan10) from that region
    product_ids = region_df.select("ProductForPlan10").unique().to_series().to_list()
    random_product = random.choice(product_ids)

    # Step 4: Filter for that product
    product_df = region_df.filter(pl.col("ProductForPlan10") == random_product)

    # Step 5: Aggregate statistics by date
    daily_stats = (
        product_df.group_by("OrderDate").agg([
            pl.col("OrderWeight").sum().alias("daily_total"),
            pl.col("OrderWeight").mean().alias("daily_mean"),
            pl.col("OrderWeight").median().alias("daily_median"),
            pl.col("OrderWeight").std().alias("daily_std"),
            pl.col("OrderWeight").max().alias("daily_max"),
            pl.col("OrderWeight").min().alias("daily_min"),
            pl.count().alias("order_count"),
        ])
        .sort("OrderDate")
    )

    # Convert to pandas for Plotly
    stats_pd = daily_stats.to_pandas()

    # Step 6: Plot using Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=stats_pd["OrderDate"],
        y=stats_pd["daily_total"],
        mode='lines+markers',
        name='Total Sales',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=stats_pd["OrderDate"],
        y=stats_pd["daily_mean"],
        mode='lines+markers',
        name='Mean Sales',
        line=dict(color='orange', dash='dash')
    ))

    # Add shaded area for ±1 standard deviation
    fig.add_trace(go.Scatter(
        x=stats_pd["OrderDate"],
        y=stats_pd["daily_mean"] + stats_pd["daily_std"],
        mode='lines',
        name='+1 Std Dev',
        line=dict(width=0),
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=stats_pd["OrderDate"],
        y=stats_pd["daily_mean"] - stats_pd["daily_std"],
        mode='lines',
        fill='tonexty',
        name='±1 Std Dev',
        fillcolor='rgba(0, 0, 255, 0.1)',
        line=dict(width=0),
    ))

    fig.update_layout(
        title=f"📦 Product {random_product} in Region {random_region}",
        xaxis_title="Order Date",
        yaxis_title="OrderWeight",
        legend=dict(x=0, y=1.1, orientation="h"),
        hovermode="x unified",
        template="plotly_white",
        margin=dict(l=40, r=40, t=60, b=40),
        height=500,
        width=1000
    )

    fig.show()


In [ ]:
visual_stat_on_plan10(transaction)

# Data Preprocess

In [ ]:
transaction1 = transaction1.drop(["key_cv_p", "WeekMon", "DeliveryDate", "WarehouseBKey", "DeliveryWeight",
                                  "price_key", "cls_prd_price", "prd_price", "__index_level_0__",
                                  "CLASS_PRICE", "CustomerArea3NameLocal",
                                  "WeightUnit", "region", 'p10_avg_price'])
''' transaction2 = transaction2.drop(["key_cv_p", "WeekMon", "DeliveryDate", "WarehouseBKey", "DeliveryWeight",
                                  "price_key", "cls_prd_price", "prd_price", "__index_level_0__",
                                  "CLASS_PRICE", "CustomerArea3NameLocal",
                                  "WeightUnit", "region", 'p10_avg_price']) '''

#"region"
#'key_cv_p'

In [ ]:
transaction1 = transaction1.group_by(['CustomerBKey', 'OrderDate', 'ProductBKey']).agg([
    pl.col('OrderWeight').sum().alias('OrderWeight'),
    pl.col('ProductForPlan1').first(),
    pl.col('ProductForPlan8').first(),
    pl.col('ProductForPlan10').first(),
    pl.col('MODEL_4').first()
])

'''transaction2 = transaction2.group_by(['CustomerBKey', 'OrderDate', 'ProductBKey']).agg([
    pl.col('OrderWeight').sum().alias('OrderWeight'),
    pl.col('ProductForPlan1').first(),
    pl.col('ProductForPlan8').first(),
    pl.col('ProductForPlan10').first(),
    pl.col('MODEL_4').first()
])'''

### Time_Filter

In [ ]:
print(f"Min date: {transaction1['OrderDate'].min()}")
print(f"Max date: {transaction1['OrderDate'].max()}")

In [ ]:
# transaction1 = transaction1.filter(
#     (pl.col("OrderDate").is_between(pl.lit("2023-01-01").cast(pl.Date), pl.lit("2023-05-01").cast(pl.Date))) |
#     (pl.col("OrderDate").is_between(pl.lit("2024-01-01").cast(pl.Date), pl.lit("2024-05-01").cast(pl.Date))) |
#     (pl.col("OrderDate").is_between(pl.lit("2025-01-01").cast(pl.Date), pl.lit("2025-04-03").cast(pl.Date)))
# )

In [ ]:
'''transaction3 = transaction2.filter(
    (pl.col("OrderDate").is_between(pl.lit("2023-02-01").cast(pl.Date), pl.lit("2023-05-01").cast(pl.Date)))
)'''

transaction2 = transaction1.filter(
    (pl.col("OrderDate").is_between(pl.lit("2024-02-01").cast(pl.Date), pl.lit("2024-05-01").cast(pl.Date)))
)

transaction1 = transaction1.filter(
    (pl.col("OrderDate").is_between(pl.lit("2025-02-01").cast(pl.Date), pl.lit("2025-04-03").cast(pl.Date)))
)

print(f"Min date1: {transaction1['OrderDate'].min()}")
print(f"Max date1: {transaction1['OrderDate'].max()}")

print(f"Min date2: {transaction2['OrderDate'].min()}")
print(f"Max date2: {transaction2['OrderDate'].max()}")

'''print(f"Min date3: {transaction3['OrderDate'].min()}")
print(f"Max date3: {transaction3['OrderDate'].max()}")'''

In [ ]:
transaction1

### Filter_people

In [ ]:
test_df = pl.read_csv('/project/ai901504-ai0004/501641_Big/week3/demand-forecasting-non-duck-product/problem1_test.csv',
                      schema_overrides={'ProductBKey': pl.Utf8})

In [ ]:
list_people = test_df['CustomerBKey'].unique()
list_item = test_df['ProductBKey'].unique()

print(transaction1.filter(
    pl.col("CustomerBKey").is_in(list_people)
).select("CustomerBKey").unique().height)

print(transaction2.filter(
    pl.col("CustomerBKey").is_in(list_people)
).select("CustomerBKey").unique().height)

'''print(transaction3.filter(
    pl.col("CustomerBKey").is_in(list_people)
).select("CustomerBKey").unique().height)'''

transaction1 = transaction1.filter(pl.col("CustomerBKey").is_in(list_people))
transaction2 = transaction2.filter(pl.col("CustomerBKey").is_in(list_people))
# transaction3 = transaction3.filter(pl.col("CustomerBKey").is_in(list_people))

### Time_series

In [ ]:
print(transaction1.columns)

In [ ]:
def to_timeseries(df_sales: pl.DataFrame) -> pl.DataFrame:
    # 1. Get min and max date
    min_date = df_sales.select(pl.col("OrderDate").min()).item()
    max_date = df_sales.select(pl.col("OrderDate").max()).item()

    # 2. Create calendar
    calendar_df = pl.DataFrame({
        "OrderDate": pl.date_range(min_date, max_date, interval="1d", eager=True)
    })

    # 3. Get frequent Customer-Product pairs
    pairs = (
        df_sales
        .group_by(["CustomerBKey", "ProductForPlan10", 'ProductForPlan1', 'ProductBKey', 'MODEL_4', 'ProductForPlan8'])
        .len()
        .filter(pl.col("len") > 5)
        .select(["CustomerBKey", "ProductForPlan10", 'ProductForPlan1', 'ProductBKey', 'MODEL_4', 'ProductForPlan8'])
    )

    # 4. Cross join pairs with dates to build time series base
    base_df = pairs.join(calendar_df, how="cross")

    sales_data = df_sales.select([
        'OrderDate', 'CustomerBKey', 'ProductForPlan10', 'OrderWeight'
    ]).with_columns(pl.col("OrderDate").cast(pl.Date))

    base_df = base_df.join(sales_data, on=["OrderDate", "CustomerBKey", "ProductForPlan10"], how="left")

    # 6. Fill missing OrderWeight with 0
    base_df = base_df.with_columns(
        pl.col("OrderWeight").fill_null(0.0).cast(pl.Float32)
    )

    return base_df

def cutout_zeros(df: pl.DataFrame, col: str = 'OrderWeight', fraction: float = 0.5, seed: int = 42) -> pl.DataFrame:
    """
    Randomly removes a fraction of rows where `col == 0`.

    Args:
        df (pl.DataFrame): The Polars DataFrame.
        col (str): Column name to evaluate (usually "sales").
        frac (float): Fraction (0.0 to 1.0) of zero-value rows to keep.
        seed (int): Random seed for reproducibility.

    Returns:
        pl.DataFrame: Filtered DataFrame with reduced zero-value rows.
    """
    np.random.seed(seed)

    # Count stats
    zero_rows = df.filter(pl.col(col) == 0)
    non_zero_rows = df.filter(pl.col(col) > 0)

    print(f"Original rows with {col} == 0: {len(zero_rows)}")
    print(f"Original rows with {col} >  0: {len(non_zero_rows)}")


    # Sample from zero rows
    n_samples = int(zero_rows.height * fraction)
    zero_sampled = zero_rows.sample(n=n_samples, seed=seed)

    # Combine back
    result = pl.concat([non_zero_rows, zero_sampled]).rechunk()
    print(f"After cutout: {result.filter(pl.col(col) == 0).height} rows with {col} == 0")
    print(f"Total rows after cutout: {result.height}")

    return result

transaction1 = to_timeseries(transaction1)
transaction1 = cutout_zeros(transaction1, col='OrderWeight', fraction=1, seed=42)

transaction2 = to_timeseries(transaction2)
transaction2 = cutout_zeros(transaction2, col='OrderWeight', fraction=1, seed=42)

'''transaction3 = to_timeseries(transaction3)
transaction3 = cutout_zeros(transaction3, col='OrderWeight', fraction=1, seed=42)'''

transaction = pl.concat([transaction1, transaction2]).rechunk()
del transaction1, transaction2

In [ ]:
import gc
gc.collect()

In [ ]:
transaction = transaction.with_columns([
    pl.col("OrderDate").dt.weekday().alias("day_of_week"),  # 0 = Monday
    pl.col("OrderDate").dt.month().alias("month"),
    pl.when(((pl.col("OrderDate").dt.day() - 1) // 7 + 1) > 4)
      .then(4)
      .otherwise((pl.col("OrderDate").dt.day() - 1) // 7 + 1)
      .alias("week"),
    pl.col("OrderDate").dt.year().alias("year")  
])

transaction = transaction.with_columns([
    pl.col("day_of_week").map_elements(lambda x: np.sin(2 * np.pi * x / 7)).alias("day_of_week_sin"),
    pl.col("day_of_week").map_elements(lambda x: np.cos(2 * np.pi * x / 7)).alias("day_of_week_cos")
])

transaction = transaction.sort(["CustomerBKey", "ProductBKey", "OrderDate"])


### People_behavior

In [ ]:
transaction.head(12)

In [ ]:
group_stats0 = transaction.group_by(['CustomerBKey', 'ProductBKey', 'month', 'year']).agg([
    # Regular statistics
    pl.col('OrderWeight').mean().alias('OrderWeight_people_last_month_mean'),
    pl.col('OrderWeight').std().alias('OrderWeight_people_last_month_std'),
    pl.col('OrderWeight').max().alias('OrderWeight_people_last_month_max'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).min().alias('OrderWeight_people_last_month_min'),

    # Statistics for OrderWeight > 0 only
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).mean().alias('OrderWeight_people_last_month_mean_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).median().alias('OrderWeight_people_last_month_median_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).std().alias('OrderWeight_people_last_month_std_gt0')
])
print(group_stats0) #stat data of people in each week at that time of the year

null_counts = group_stats0.select([
    pl.col(col).is_null().sum().alias(col) for col in group_stats0.columns
])
print(null_counts)

group_stats0 = group_stats0.with_columns([
    pl.col("OrderWeight_people_last_month_mean").fill_null(0),
    pl.col("OrderWeight_people_last_month_std").fill_null(0),
    pl.col("OrderWeight_people_last_month_max").fill_null(0),
    pl.col("OrderWeight_people_last_month_min").fill_null(0),
    pl.col("OrderWeight_people_last_month_mean_gt0").fill_null(0),
    pl.col("OrderWeight_people_last_month_median_gt0").fill_null(0),
    pl.col("OrderWeight_people_last_month_std_gt0").fill_null(0)
])

group_stats0 = group_stats0.with_columns(
    (pl.col("month") + 1).alias("month")
)

def apply_stats0(df: pl.DataFrame, group_stats: pl.DataFrame) -> pl.DataFrame:
    df = df.join(group_stats, on=['CustomerBKey', 'ProductBKey', 'month', 'year'], how='left')

    # Optional: Fill nulls after join if any (could result from missing group_stats rows)
    df = df.fill_null(0)

    return df

transaction = apply_stats0(transaction, group_stats0)
print(transaction.head(4))

In [ ]:
group_stats4 = transaction.group_by(['CustomerBKey', 'ProductBKey', 'month', 'year', 'week']).agg([
    # Regular statistics
    pl.col('OrderWeight').mean().alias('OrderWeight_people_last_month_week_mean'),
    pl.col('OrderWeight').std().alias('OrderWeight_people_last_month_week_std'),
    pl.col('OrderWeight').max().alias('OrderWeight_people_last_month_week_max'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).min().alias('OrderWeight_people_last_month_week_min'),

    # Statistics for OrderWeight > 0 only
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).mean().alias('OrderWeight_people_last_month_week_mean_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).median().alias('OrderWeight_people_last_month_week_median_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).std().alias('OrderWeight_people_last_month_week_std_gt0')
])
print(group_stats4) #stat data of people in each week at that time of the year

null_counts = group_stats4.select([
    pl.col(col).is_null().sum().alias(col) for col in group_stats4.columns
])
print(null_counts)

group_stats4 = group_stats4.with_columns([
    pl.col("OrderWeight_people_last_month_week_mean").fill_null(0),
    pl.col("OrderWeight_people_last_month_week_std").fill_null(0),
    pl.col("OrderWeight_people_last_month_week_max").fill_null(0),
    pl.col("OrderWeight_people_last_month_week_min").fill_null(0),
    pl.col("OrderWeight_people_last_month_week_mean_gt0").fill_null(0),
    pl.col("OrderWeight_people_last_month_week_median_gt0").fill_null(0),
    pl.col("OrderWeight_people_last_month_week_std_gt0").fill_null(0)
])

group_stats4 = group_stats4.with_columns(
    (pl.col("month") + 1).alias("month")
)

group_stats4 = group_stats4.with_columns([
    ((pl.col("week") % 4) + 1).alias("week"),  # Shift week forward, wrap around after 4
    (pl.when(pl.col("week") == 1)
     .then(pl.col("month") + 1)
     .otherwise(pl.col("month"))).alias("month")
])

def apply_stats4(df: pl.DataFrame, group_stats: pl.DataFrame) -> pl.DataFrame:
    df = df.join(group_stats, on=['CustomerBKey', 'ProductBKey', 'month', 'year', 'week'], how='left')

    # Drop rows where 'month' is 2 (February) because there's no previous month data (i.e., no stats from Jan)
    df = df.filter(pl.col("month") != 2)

    # Optional: Fill nulls after join if any (could result from missing group_stats rows)
    df = df.fill_null(0)

    return df

transaction = apply_stats4(transaction, group_stats4)
print(transaction.head(4))

In [ ]:
group_stats1 = transaction.group_by(['CustomerBKey', 'ProductForPlan1', 'ProductForPlan8', 'month', 'week']).agg([
    # Regular statistics
    pl.col('OrderWeight').mean().alias('OrderWeight_people_mean'),
    pl.col('OrderWeight').median().alias('OrderWeight_people_median'),
    pl.col('OrderWeight').std().alias('OrderWeight_people_std'),
    pl.col('OrderWeight').max().alias('OrderWeight_people_max'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).min().alias('OrderWeight_people_min'),

    # Statistics for OrderWeight > 0 only
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).mean().alias('OrderWeight_people_mean_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).median().alias('OrderWeight_people_median_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).std().alias('OrderWeight_people_std_gt0')
])
print(group_stats1) #stat data of people in each week at that time of the year

In [ ]:
null_counts = group_stats1.select([
    pl.col(col).is_null().sum().alias(col) for col in group_stats1.columns
])
print(null_counts)
group_stats1 = group_stats1.with_columns([
    pl.col("OrderWeight_people_std").fill_null(0),
    pl.col("OrderWeight_people_min").fill_null(0),
    pl.col("OrderWeight_people_mean_gt0").fill_null(0),
    pl.col("OrderWeight_people_median_gt0").fill_null(0),
    pl.col("OrderWeight_people_std_gt0").fill_null(0)
])

In [ ]:
def apply_stats1(df: pl.DataFrame, group_stats: pl.DataFrame) -> pl.DataFrame:
    df = df.join(group_stats, on=['CustomerBKey', 'ProductForPlan1', 'ProductForPlan8', 'month', 'week'], how='left')
    return df

transaction = apply_stats1(transaction, group_stats1)
print(transaction.head(4))

In [ ]:
group_stats2 = transaction.group_by(['CustomerBKey', 'ProductForPlan1', 'ProductForPlan8', 'month']).agg([
    # Regular statistics
    pl.col('OrderWeight').mean().alias('OrderWeight_people_mean_inmonth'),
    pl.col('OrderWeight').median().alias('OrderWeight_people_median_inmonth'),
    pl.col('OrderWeight').std().alias('OrderWeight_people_std_inmonth')
])
print(group_stats2) #stat data of people in each week at that time of the year

In [ ]:
null_counts = group_stats2.select([
    pl.col(col).is_null().sum().alias(col) for col in group_stats2.columns
])
print(null_counts)
group_stats2 = group_stats2.with_columns([
    pl.col("OrderWeight_people_mean_inmonth").fill_null(0),
    pl.col("OrderWeight_people_median_inmonth").fill_null(0),
    pl.col("OrderWeight_people_std_inmonth").fill_null(0)
])

In [ ]:
def apply_stats2(df: pl.DataFrame, group_stats: pl.DataFrame) -> pl.DataFrame:
    df = df.join(group_stats, on=['CustomerBKey', 'ProductForPlan1', 'ProductForPlan8', 'month'], how='left')
    return df

transaction = apply_stats2(transaction, group_stats2)
print(transaction.head(4))

In [ ]:
group_stats3 = transaction.group_by(['CustomerBKey', 'ProductBKey', 'day_of_week']).agg([
    # Regular statistics
    pl.col('OrderWeight').mean().alias('OrderWeight_people_cycle_mean'),
    pl.col('OrderWeight').median().alias('OrderWeight_people_cycle_median'),
    pl.col('OrderWeight').std().alias('OrderWeight_people_cycle_std'),
    pl.col('OrderWeight').max().alias('OrderWeight_people_cycle_max'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).min().alias('OrderWeight_people_cycle_min'),

    # Statistics for OrderWeight > 0 only
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).mean().alias('OrderWeight_people_cycle_mean_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).median().alias('OrderWeight_people_cycle_median_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).std().alias('OrderWeight_people_cycle_std_gt0')
])
print(group_stats3) #stat data of people in each week at that time of the year

In [ ]:
null_counts = group_stats3.select([
    pl.col(col).is_null().sum().alias(col) for col in group_stats3.columns
])
print(null_counts)
group_stats3 = group_stats3.with_columns([
    pl.col("OrderWeight_people_cycle_mean").fill_null(0),
    pl.col("OrderWeight_people_cycle_median").fill_null(0),
    pl.col("OrderWeight_people_cycle_std").fill_null(0),
    pl.col("OrderWeight_people_cycle_max").fill_null(0),
    pl.col("OrderWeight_people_cycle_min").fill_null(0),
    pl.col("OrderWeight_people_cycle_mean_gt0").fill_null(0),
    pl.col("OrderWeight_people_cycle_median_gt0").fill_null(0),
    pl.col("OrderWeight_people_cycle_std_gt0").fill_null(0)
])

In [ ]:
def apply_stats3(df: pl.DataFrame, group_stats: pl.DataFrame) -> pl.DataFrame:
    df = df.join(group_stats, on=['CustomerBKey', 'ProductBKey', 'day_of_week'], how='left')
    return df

transaction = apply_stats3(transaction, group_stats3)
print(transaction.head(4))

### Statistic data

In [ ]:
transaction

In [ ]:
group_stats = transaction.group_by(['ProductForPlan10', 'MODEL_4', 'month', 'week']).agg([
    # Regular statistics
    pl.col('OrderWeight').mean().alias('OrderWeight_mean'),
    pl.col('OrderWeight').median().alias('OrderWeight_median'),
    pl.col('OrderWeight').std().alias('OrderWeight_std'),
    pl.col('OrderWeight').max().alias('OrderWeight_max'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).min().alias('OrderWeight_min'),

    # Statistics for OrderWeight > 0 only
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).mean().alias('OrderWeight_mean_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).median().alias('OrderWeight_median_gt0'),
    pl.col('OrderWeight').filter(pl.col('OrderWeight') > 0).std().alias('OrderWeight_std_gt0')
])

In [ ]:
group_stats

In [ ]:
null_counts = group_stats.select([
    pl.col(col).is_null().sum().alias(col) for col in group_stats.columns
])
print(null_counts)

In [ ]:
group_stats = group_stats.with_columns([
    pl.col("OrderWeight_std").fill_null(0),
    pl.col("OrderWeight_min").fill_null(0),
    pl.col("OrderWeight_mean_gt0").fill_null(0),
    pl.col("OrderWeight_median_gt0").fill_null(0),
    pl.col("OrderWeight_std_gt0").fill_null(0)
])

In [ ]:
def apply_stats(df: pl.DataFrame, group_stats: pl.DataFrame) -> pl.DataFrame:
    df = df.join(group_stats, on=['ProductForPlan10', 'MODEL_4', 'month', 'week'], how='left')
    return df

transaction = apply_stats(transaction, group_stats)
print(transaction.head(4))

### Holiday

In [ ]:
event_date #newyear, songkarn, thudjene, makabusha

In [ ]:
def apply_event(df: pl.DataFrame, event_date: pl.DataFrame) -> pl.DataFrame:
    # Select only the needed columns
    event_date = (
        event_date
        .select([
            pl.col("date").cast(pl.Date).alias("OrderDate"),
            "is_songkarn",
            "is_makabusha"
        ])
    )
    df = df.join(event_date, on="OrderDate", how="left")
    return df

transaction = apply_event(transaction, event_date)

#newyear, songkarn, thudjene, makabusha

# Train

In [ ]:
print(transaction.columns)

In [ ]:
transaction

### Load_Test

In [ ]:
test_df.head(5)

In [ ]:
test_df = test_df.with_columns(
    pl.col("OrderDate").str.strptime(pl.Datetime, "%Y-%m-%d", strict=False).alias("OrderDate")
)

test_df = test_df.with_columns([
    pl.col("OrderDate").dt.weekday().alias("day_of_week"),  # 0 = Monday
    pl.col("OrderDate").dt.month().alias("month"),
    pl.when(((pl.col("OrderDate").dt.day() - 1) // 7 + 1) > 4)
      .then(4)
      .otherwise((pl.col("OrderDate").dt.day() - 1) // 7 + 1)
      .alias("week"),
    pl.col("OrderDate").dt.year().alias("year")  
])

test_df = test_df.with_columns([
    pl.col("day_of_week").map_elements(lambda x: np.sin(2 * np.pi * x / 7)).alias("day_of_week_sin"),
    pl.col("day_of_week").map_elements(lambda x: np.cos(2 * np.pi * x / 7)).alias("day_of_week_cos")
])


In [ ]:
def add_column_from_join(
    left_df: pl.DataFrame,
    right_df: pl.DataFrame,
    join_keys: list[str],
    target_column: str,
    how: str = "left"
) -> pl.DataFrame:
    """
    Join left_df with right_df on join_keys, adding only target_column from right_df.

    Args:
        left_df (pl.DataFrame): The main DataFrame.
        right_df (pl.DataFrame): The DataFrame to join from.
        join_keys (list[str]): Columns to join on (must exist in both).
        target_column (str): Column name to add from right_df.
        how (str, optional): Join type. Defaults to "left".

    Returns:
        pl.DataFrame: Resulting DataFrame with target_column added.
    """
    right_trimmed = right_df.select(join_keys + [target_column]).unique(subset=join_keys)
    return left_df.join(right_trimmed, on=join_keys, how=how)

In [ ]:
test_df = add_column_from_join(
    left_df=test_df,
    right_df=transaction,
    join_keys=["ProductForPlan10", "ProductForPlan1"],
    target_column="ProductForPlan8"
)

test_df = apply_stats0(test_df, group_stats0)
test_df = apply_stats4(test_df, group_stats4)
test_df = apply_stats(test_df, group_stats)
test_df = apply_stats1(test_df, group_stats1)
test_df = apply_stats2(test_df, group_stats2)
test_df = apply_stats3(test_df, group_stats3)

test_df = test_df.with_columns(pl.col("OrderDate").cast(pl.Date))
test_df = apply_event(test_df, event_date)

In [ ]:
test_df = test_df.to_pandas()

print(test_df.info())
print(f"ProductForPlan1 unique count: {test_df['ProductForPlan1'].nunique()}")
print(f"CustomerBKey unique count: {test_df['CustomerBKey'].nunique()}")
print(f"ProductForPlan10 unique count: {test_df['ProductForPlan10'].nunique()}")
print(f"MODEL_4 unique count: {test_df['MODEL_4'].nunique()}")
print(test_df.isnull().sum())

In [ ]:
# Get all columns except 'OrderWeight'
cols_to_fill = test_df.columns.difference(['OrderWeight'])

# Fill NaNs in those columns with 0
test_df[cols_to_fill] = test_df[cols_to_fill].fillna(0)

In [ ]:
train_df = transaction.to_pandas()

In [ ]:
print(train_df.info())
print(f"ProductForPlan1 unique count: {train_df['ProductForPlan1'].nunique()}")
print(f"ProductForPlan10 unique count: {train_df['ProductForPlan10'].nunique()}")
print(f"MODEL_4 unique count: {train_df['MODEL_4'].nunique()}")
print(train_df.isnull().sum())

In [ ]:
cat_columns = ['ProductForPlan1', 'ProductBKey', 'MODEL_4', 'ProductForPlan8',
    'is_songkarn', 'is_makabusha', 'CustomerBKey',
    'day_of_week', 'month', 'week', 'ProductForPlan10']
train_df[cat_columns] = train_df[cat_columns].astype('category')
test_df[cat_columns] = test_df[cat_columns].astype('category')

train_df = train_df.drop("year", axis=1)
test_df = test_df.drop("year", axis=1)

### LightGBM

In [ ]:
print(train_df.columns)

#### LGBMbyRegion

In [ ]:
import lightgbm as lgb # Make sure you import LGBMClassifier too
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, log_loss, accuracy_score, mean_absolute_error # Import classification metrics
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.metrics import roc_curve


def autotrain(df, test_df): # Removed unused 'feature' parameter
    region = df['MODEL_4'].unique()
    model_list = []
    all_predictions = []
    feature_importances_records = []
    classifier_list = []

    target = 'OrderWeight'
    
    base_features = ['CustomerBKey', 'ProductForPlan10', 'ProductForPlan1', 'ProductBKey',
       'ProductForPlan8', 'day_of_week',
       'month', 'week', 'OrderWeight_people_mean', 'OrderWeight_people_median',
       'OrderWeight_people_std', 'OrderWeight_people_max',
       'OrderWeight_people_min', 'OrderWeight_people_mean_gt0',
       'OrderWeight_people_median_gt0', 'OrderWeight_people_std_gt0',
       'OrderWeight_people_mean_inmonth', 'OrderWeight_people_median_inmonth',
       'OrderWeight_people_std_inmonth', 'OrderWeight_people_cycle_mean', 
       'OrderWeight_people_cycle_median', 'OrderWeight_people_cycle_std', 
       'OrderWeight_people_cycle_max', 'OrderWeight_people_cycle_min', 
       'OrderWeight_people_cycle_mean_gt0', 'OrderWeight_people_cycle_median_gt0', 
       'OrderWeight_people_cycle_std_gt0', 
       'OrderWeight_mean', 'OrderWeight_median', 'OrderWeight_std', 'OrderWeight_max',
       'OrderWeight_min', 'OrderWeight_mean_gt0', 'OrderWeight_median_gt0',
       'OrderWeight_std_gt0', 'is_new_year', 'is_songkarn', 'is_thudjene',
       'is_makabusha']
    
    categorical_features = [
    'CustomerBKey', 'ProductForPlan10', 'day_of_week', 'month', 'week', 
    'ProductForPlan1', 'ProductBKey', 'ProductForPlan8',
    'is_new_year', 'is_songkarn', 'is_thudjene', 'is_makabusha', 'CustomerBKey'
    ]
    # Add your engineered features here if you have them!

    # --- Step 1: Create the binary target 'has_sales' ---
    df['has_sales'] = (df[target] > 0).astype(int)
    # test_df does not have 'units_sold', so 'has_sales' cannot be created directly.
    # The classifier's prediction will provide this information for test_df.

    # --- Define a fixed random state for reproducibility ---
    FIXED_RANDOM_STATE = 42

    for region_pred in region:
        print(f"\nProcessing Region {region_pred}...")
        region_df = df[df['MODEL_4'] == region_pred].copy()
        region_test_df = test_df[test_df['MODEL_4'] == region_pred].copy()

        current_features = base_features # Use the defined base features

        missing = [col for col in current_features if col not in region_df.columns]
        if missing:
            print(f"Store {region_pred} missing features: {missing}")
            continue

        X_region = region_df[current_features]
        y_sales_region = region_df[target]
        y_has_sales_region = region_df['has_sales'] # Use the binary target

        # --- Step 2a: Train a Classifier to predict has_sales ---
        print(f"  Training classifier for region {region_pred}...")
        # Split ALL data for this store for classifier training and validation
        X_train_clf, X_val_clf, y_train_has_sales, y_val_has_sales = train_test_split(
            X_region, y_has_sales_region, test_size=0.2, shuffle=True, stratify=y_has_sales_region,
            random_state=FIXED_RANDOM_STATE # <-- Added fixed random state
        )

        classifier = LGBMClassifier(
            objective='binary',          # Binary classification objective
            metric='logloss',            # Evaluation metric
            n_estimators=150,
            learning_rate=0.05,
            num_leaves=63,               # Default LightGBM value, often good
            verbose=-1,                  # Suppress verbose output
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1                    # Use all available cores
        )

        classifier.fit(
            X_train_clf,
            y_train_has_sales,
            eval_set=[(X_val_clf, y_val_has_sales)],
            eval_metric='logloss',
            categorical_feature=categorical_features,
            callbacks=[lgb.early_stopping(10, verbose=False)] # Add early stopping
        )

        val_probs = classifier.predict_proba(X_val_clf)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_val_has_sales, val_probs)
        optimal_threshold = thresholds[np.argmax(tpr - fpr)] + 0.02

        print(f"  Region {region_pred} Optimal Threshold: {optimal_threshold:.4f}")

        # Predict probability of having sales (class 1) for ALL instances in store_df and store_test_df
        # This probability will be used as a feature and for the final prediction logic
        region_df['prob_has_sales'] = classifier.predict_proba(X_region)[:, 1]
        region_test_df['prob_has_sales'] = classifier.predict_proba(region_test_df[current_features])[:, 1]

        clf_logloss = log_loss(y_val_has_sales, classifier.predict_proba(X_val_clf)[:, 1])
        print(f"  Region {region_pred} Classifier Metrics | LogLoss: {clf_logloss:.4f}") # Added accuracy optional above

        # --- Step 2b: Train the Regressor (Option 1: on ALL data) ---
        # --- Step 2b: Train the Regressor (LightGBM) ---
        print(f"  Training regressor for region {region_pred} on ALL data...")

        # Define features for the regressor (base features + classifier probability)
        regressor_features = current_features + ['prob_has_sales']

        # Split data for regressor training and validation
        X_reg = region_df[regressor_features]
        y_reg = region_df[target]

        X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(
            X_reg, y_reg, test_size=0.2, shuffle=True,
            random_state=FIXED_RANDOM_STATE
        )

        regressor = LGBMRegressor(
            objective='tweedie',
            tweedie_variance_power=1.5,
            n_estimators=200,
            learning_rate=0.05,
            max_depth=8,
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1,
            eval_metric='mae',
            verbose=-1
        )

        regressor.fit(
            X_train_reg,
            y_train_reg,
            eval_set=[(X_val_reg, y_val_reg)],
            categorical_feature=categorical_features,
            callbacks=[lgb.early_stopping(10, verbose=False)]
        )

        print("  Regressor training complete.")

        # --- Get Feature Importances for the Regressor ---
        importances = regressor.feature_importances_
        for feature_name, importance_value in zip(regressor_features, importances):
            feature_importances_records.append({
                'store_id': region_pred,
                'feature_name': feature_name,
                'importance_value': importance_value
            })

        importance_df = pd.DataFrame({
            'feature': regressor_features,
            'importance': importances
        }).sort_values(by='importance', ascending=False)

        top5 = importance_df.head(5)
        print(f"  Top 5 important features for region {region_pred}:")
        print(top5.to_string(index=False))

        val_reg_pred = regressor.predict(X_val_reg)

        # Combine with classifier probability
        final_val_pred = np.where(
            X_val_reg['prob_has_sales'] > optimal_threshold,
            val_reg_pred,
            0
        )
        final_val_pred[final_val_pred < 0] = 0

        mae_score = mean_absolute_error(y_val_reg, final_val_pred)

        print(f"  Region {region_pred} MAE: {mae_score:.4f}")

        # --- Predict on Test Data ---
        if region_test_df.empty:
            print(f"  No test data for region {region_pred}.")
            continue

        # store_test_df already has 'prob_has_sales' added earlier
        X_test_combined = region_test_df[regressor_features] # Use the same regressor features

        # Predict sales amount using the regressor on the test data
        y_pred_test_reg = regressor.predict(X_test_combined)

        # Apply the classification threshold logic to test predictions
        y_pred_test_final = np.where(
            X_test_combined['prob_has_sales'] > optimal_threshold, # Use the predicted probability from the classifier
            y_pred_test_reg,
            0 # Predict 0 if probability is below threshold
        )

        # Ensure final test predictions are non-negative
        y_pred_test_final[y_pred_test_final < 0] = 0


        region_test_df['predicted_sales'] = y_pred_test_final
        all_predictions.append(region_test_df[['id', 'predicted_sales']])

    # --- Consolidate predictions and merge back to test_df outside the loop ---
    if all_predictions:
      if 'predicted_sales' in test_df.columns:
        test_df = test_df.drop(columns=['predicted_sales'])

      prediction_df = pd.concat(all_predictions, ignore_index=True)

      # Ensure 'id' exists and matches in test_df and prediction_df
      test_df = test_df.merge(prediction_df, on='id', how='left')

      # Fill missing predictions with 0
      test_df['predicted_sales'] = test_df['predicted_sales'].fillna(0)


    return model_list, test_df, feature_importances_records

In [ ]:
feature_importances_records = []
model_list, test_df, feature_importances_records = autotrain(train_df, test_df)

#### LightGBMbyItem

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, log_loss
from xgboost import XGBRegressor
from xgboost.callback import EarlyStopping
from sklearn.metrics import roc_curve

def autotrain2(df, test_df):
    items = df['ProductForPlan10'].unique()
    model_list = []
    all_predictions = []
    feature_importances_records = []

    target = 'OrderWeight'
    base_features = ['CustomerBKey', 'ProductForPlan1', 'ProductBKey',
       'ProductForPlan8', 'day_of_week', 'MODEL_4',
       'month', 'week', 'OrderWeight_people_mean', 'OrderWeight_people_median',
       'OrderWeight_people_std', 'OrderWeight_people_max',
       'OrderWeight_people_min', 'OrderWeight_people_mean_gt0',
       'OrderWeight_people_median_gt0', 'OrderWeight_people_std_gt0',
       'OrderWeight_people_mean_inmonth', 'OrderWeight_people_median_inmonth',
       'OrderWeight_people_std_inmonth', 'OrderWeight_mean',
       'OrderWeight_median', 'OrderWeight_std', 'OrderWeight_max',
       'OrderWeight_min', 'OrderWeight_mean_gt0', 'OrderWeight_median_gt0',
       'OrderWeight_std_gt0', 'is_new_year', 'is_songkarn', 'is_thudjene',
       'is_makabusha']
    
    categorical_features = [
    'CustomerBKey', 'MODEL_4', 'day_of_week', 'month', 'week', 
    'ProductForPlan1', 'ProductBKey', 'ProductForPlan8',
    'is_new_year', 'is_songkarn', 'is_thudjene', 'is_makabusha', 'CustomerBKey'
    ]


    # Create binary target
    df['has_sales'] = (df[target] > 0).astype(int)

    FIXED_RANDOM_STATE = 42

    for item in items:
        print(f"\nProcessing Item {item}...")
        item_df = df[df['ProductForPlan10'] == item].copy()
        item_test_df = test_df[test_df['ProductForPlan10'] == item].copy()

        current_features = base_features

        missing = [col for col in current_features if col not in item_df.columns]
        if missing:
            print(f"Item {item} missing features: {missing}")
            continue

        X_item = item_df[current_features]
        y_sales_item = item_df[target]
        y_has_sales_item = item_df['has_sales']

        # Train classifier
        print(f"  Training classifier for item {item}...")
        X_train_clf, X_val_clf, y_train_clf, y_val_clf = train_test_split(
            X_item, y_has_sales_item, test_size=0.2, stratify=y_has_sales_item,
            random_state=FIXED_RANDOM_STATE
        )

        classifier = LGBMClassifier(
            objective='binary',
            metric='logloss',
            n_estimators=150,
            learning_rate=0.05,
            num_leaves=31,
            verbose=-1,
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1
        )

        classifier.fit(
            X_train_clf, y_train_clf,
            eval_set=[(X_val_clf, y_val_clf)],
            eval_metric='logloss',
            categorical_feature=categorical_features,
            callbacks=[lgb.early_stopping(10, verbose=False)]
        )

        val_probs = classifier.predict_proba(X_val_clf)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_val_clf, val_probs)
        optimal_threshold = thresholds[np.argmax(tpr - fpr)]

        print(f"  Item {item} Optimal Threshold: {optimal_threshold:.4f}")

        # Apply threshold to full item_df and test set
        item_df['prob_has_sales'] = classifier.predict_proba(X_item)[:, 1]
        item_test_df['prob_has_sales'] = classifier.predict_proba(item_test_df[current_features])[:, 1]


        clf_logloss = log_loss(y_val_clf, classifier.predict_proba(X_val_clf)[:, 1])
        print(f"  Item {item} Classifier LogLoss: {clf_logloss:.4f}")

        # Train regressor
        print(f"  Training regressor for item {item}...")
        regressor_features = current_features + ['prob_has_sales']

        X_reg = item_df[regressor_features]
        y_reg = item_df[target]

        X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(
            X_reg, y_reg, test_size=0.2, random_state=FIXED_RANDOM_STATE
        )

        regressor = LGBMRegressor(
            objective='tweedie',
            tweedie_variance_power=1.5,
            n_estimators=200,
            learning_rate=0.1,
            max_depth=6,
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1,
            eval_metric='mae',
            verbose=-1
        )

        regressor.fit(
            X_train_reg,
            y_train_reg,
            eval_set=[(X_val_reg, y_val_reg)],
            categorical_feature=categorical_features,
            callbacks=[lgb.early_stopping(10, verbose=False)]
        )

        importances = regressor.feature_importances_
        for feature_name, importance_value in zip(regressor_features, importances):
            feature_importances_records.append({
                'item_id': item,
                'feature_name': feature_name,
                'importance_value': importance_value
            })

        importance_df = pd.DataFrame({
            'feature': regressor_features,
            'importance': importances
        }).sort_values(by='importance', ascending=False)

        top5 = importance_df.head(5)
        print(f"  Top 5 important features for item {item}:")
        print(top5.to_string(index=False))

        val_pred_reg = regressor.predict(X_val_reg)
        final_val_pred = np.where(
            X_val_reg['prob_has_sales'] > optimal_threshold,
            val_pred_reg,
            0
        )
        final_val_pred[final_val_pred < 0] = 0

        mae_score = mean_absolute_error(y_val_reg, final_val_pred)
        print(f"  Item {item} MAE: {mae_score:.4f}")

        # Predict on test set
        if item_test_df.empty:
            print(f"  No test data for item {item}.")
            continue

        X_test_combined = item_test_df[regressor_features]
        y_pred_test_reg = regressor.predict(X_test_combined)
        y_pred_test_final = np.where(
            X_test_combined['prob_has_sales'] > optimal_threshold,
            y_pred_test_reg,
            0
        )
        y_pred_test_final[y_pred_test_final < 0] = 0

        item_test_df['predicted_sales'] = y_pred_test_final
        all_predictions.append(item_test_df[['id', 'predicted_sales']])

    # Combine predictions
    if all_predictions:
        if 'predicted_sales' in test_df.columns:
            test_df = test_df.drop(columns=['predicted_sales'])
        prediction_df = pd.concat(all_predictions, ignore_index=True)
        test_df = test_df.merge(prediction_df, on='id', how='left')
        test_df['predicted_sales'] = test_df['predicted_sales'].fillna(0)

    return model_list, test_df, feature_importances_records


In [ ]:
feature_importances_records = []
model_list, test_df, feature_importances_records = autotrain2(train_df, test_df)

#### XGBOOST

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, log_loss
from xgboost import XGBRegressor, XGBClassifier
from xgboost.callback import EarlyStopping
from sklearn.metrics import roc_curve

def autotrain3(df, test_df):
    items = df['ProductForPlan10'].unique()
    model_list = []
    all_predictions = []
    feature_importances_records = []

    target = 'OrderWeight'
    base_features = ['CustomerBKey', 'ProductForPlan1', 'ProductBKey', 
                     'MODEL_4', 'ProductForPlan8', 'day_of_week', 
                     'month', 'week', 'OrderWeight_people_mean', 'OrderWeight_people_median', 
                     'OrderWeight_people_std', 'OrderWeight_people_max', 'OrderWeight_people_min', 
                     'OrderWeight_people_last_month_mean', 'OrderWeight_people_last_month_std', 
                     'OrderWeight_people_last_month_max', 'OrderWeight_people_last_month_min', 
                     'OrderWeight_people_last_month_mean_gt0', 'OrderWeight_people_last_month_median_gt0', 
                     'OrderWeight_people_last_month_std_gt0',
                     'OrderWeight_people_mean_gt0', 'OrderWeight_people_median_gt0', 
                     'OrderWeight_people_std_gt0', 'OrderWeight_people_mean_inmonth', 
                     'OrderWeight_people_median_inmonth', 'OrderWeight_people_std_inmonth', 
                     'OrderWeight_people_cycle_mean', 'OrderWeight_people_cycle_median', 
                     'OrderWeight_people_cycle_std', 'OrderWeight_people_cycle_max', 
                     'OrderWeight_people_cycle_min', 'OrderWeight_people_cycle_mean_gt0', 
                     'OrderWeight_people_cycle_median_gt0', 'OrderWeight_people_cycle_std_gt0', 
                     'OrderWeight_mean', 'OrderWeight_median', 'OrderWeight_std', 'OrderWeight_max', 
                     'OrderWeight_min', 'OrderWeight_mean_gt0', 'OrderWeight_median_gt0',
                     'OrderWeight_people_last_month_week_mean',          
                     'OrderWeight_people_last_month_week_std',           
                     'OrderWeight_people_last_month_week_max',
                     'OrderWeight_people_last_month_week_min',           
                     'OrderWeight_people_last_month_week_mean_gt0',
                     "OrderWeight_people_last_month_week_median_gt0",
                     'OrderWeight_people_last_month_week_std_gt0',
                     'OrderWeight_std_gt0', 'is_songkarn', 'is_makabusha',
                     "day_of_week_sin", "day_of_week_cos"]
    
    categorical_features = [
    'ProductForPlan1', 'ProductBKey', 'MODEL_4', 'ProductForPlan8',
    'is_songkarn', 'is_makabusha', 'CustomerBKey',
    'day_of_week', 'month', 'week',
    ]


    # Create binary target
    df['has_sales'] = (df[target] > 0).astype(int)

    FIXED_RANDOM_STATE = 42

    for item in items:
        print(f"\nProcessing Item {item}...")
        item_df = df[df['ProductForPlan10'] == item].copy()
        item_test_df = test_df[test_df['ProductForPlan10'] == item].copy()

        current_features = base_features

        missing = [col for col in current_features if col not in item_df.columns]
        if missing:
            print(f"Item {item} missing features: {missing}")
            continue

        X_item = item_df[current_features]
        y_sales_item = item_df[target]
        y_has_sales_item = item_df['has_sales']

        # Train classifier
        print(f"  Training classifier for item {item}...")
        X_train_clf, X_val_clf, y_train_clf, y_val_clf = train_test_split(
            X_item, y_has_sales_item, test_size=0.2, stratify=y_has_sales_item,
            random_state=FIXED_RANDOM_STATE
        )

        classifier = LGBMClassifier(
            objective='binary',
            metric='binary_logloss',
            n_estimators=200,
            learning_rate=0.05,
            num_leaves=64,
            max_depth=10,
            verbose=-1,
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1
        )

        classifier.fit(
            X_train_clf, y_train_clf,
            eval_set=[(X_val_clf, y_val_clf)],
            eval_metric='logloss',
            categorical_feature=categorical_features,
            callbacks=[lgb.early_stopping(10, verbose=False)]
        )

        val_probs = classifier.predict_proba(X_val_clf)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_val_clf, val_probs)
        optimal_threshold = thresholds[np.argmax(tpr - fpr)] + 0.05

        print(f"  Item {item} Optimal Threshold: {optimal_threshold:.4f}")

        # Apply threshold to full item_df and test set
        item_df['prob_has_sales'] = classifier.predict_proba(X_item)[:, 1]
        item_test_df['prob_has_sales'] = classifier.predict_proba(item_test_df[current_features])[:, 1]


        clf_logloss = log_loss(y_val_clf, classifier.predict_proba(X_val_clf)[:, 1])
        print(f"  Item {item} Classifier LogLoss: {clf_logloss:.4f}")

        # Train regressor
        print(f"  Training regressor for item {item}...")
        regressor_features = current_features + ['prob_has_sales']

        X_reg = item_df[regressor_features]
        y_reg = item_df[target]

        X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(
            X_reg, y_reg, test_size=0.2, random_state=FIXED_RANDOM_STATE
        )

        regressor = XGBRegressor(
            objective='reg:tweedie',
            tweedie_variance_power=1.5,
            n_estimators=200,
            learning_rate=0.05,
            max_depth=12,
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1,
            eval_metric='mae',
            verbosity=0,
            enable_categorical=True
        )

        # Removed: early_stopping_callback_xgb definition
        regressor.fit(
            X_train_reg,
            y_train_reg,
            # Removed: eval_set and callbacks arguments from regressor.fit()
        )

        importances = regressor.feature_importances_
        for feature_name, importance_value in zip(regressor_features, importances):
            feature_importances_records.append({
                'item_id': item,
                'feature_name': feature_name,
                'importance_value': importance_value
            })

        importance_df = pd.DataFrame({
            'feature': regressor_features,
            'importance': importances
        }).sort_values(by='importance', ascending=False)

        top5 = importance_df.head(5)
        print(f"  Top 5 important features for item {item}:")
        print(top5.to_string(index=False))

        val_pred_reg = regressor.predict(X_val_reg)
        final_val_pred = np.where(
            X_val_reg['prob_has_sales'] > optimal_threshold,
            val_pred_reg,
            0
        )
        final_val_pred[final_val_pred < 0] = 0

        mae_score = mean_absolute_error(y_val_reg, final_val_pred)
        print(f"  Item {item} MAE: {mae_score:.4f}")

        # Predict on test set
        if item_test_df.empty:
            print(f"  No test data for item {item}.")
            continue

        X_test_combined = item_test_df[regressor_features]
        y_pred_test_reg = regressor.predict(X_test_combined)
        y_pred_test_final = np.where(
            X_test_combined['prob_has_sales'] > optimal_threshold,
            y_pred_test_reg,
            0
        )
        y_pred_test_final[y_pred_test_final < 0] = 0

        item_test_df['predicted_sales'] = y_pred_test_final
        all_predictions.append(item_test_df[['id', 'predicted_sales']])

    # Combine predictions
    if all_predictions:
        prediction_df = pd.concat(all_predictions, ignore_index=True)
        test_df = test_df.merge(prediction_df, on='id', how='left')
        test_df['predicted_sales'] = test_df['predicted_sales'].fillna(0)

    return model_list, test_df, feature_importances_records


In [ ]:
feature_importances_records = []
model_list, test_df, feature_importances_records = autotrain3(train_df, test_df)

### Autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_df = train_df.drop(columns=['day_of_week', 'week', 'month'])
test_df = test_df.drop(columns=['day_of_week', 'week', 'month'])

In [ ]:
predictor = TabularPredictor(label='OrderWeight').fit(
    train_data=train_df,
    num_gpus=1,
    presets='good'
)


In [ ]:
y_pred = predictor.predict(test_df.drop(columns=['OrderWeight']))

In [ ]:
y_pred.head()

In [ ]:
test_df['OrderWeight'] = y_pred

In [ ]:
# Drop day_of_week and month
test_df = test_df.drop(columns=['ProductForPlan8',
                                'OrderWeight_people_mean', 'OrderWeight_people_median', 
                     'OrderWeight_people_std', 'OrderWeight_people_max', 'OrderWeight_people_min', 
                     'OrderWeight_people_last_month_mean', 'OrderWeight_people_last_month_std', 
                     'OrderWeight_people_last_month_max', 'OrderWeight_people_last_month_min', 
                     'OrderWeight_people_last_month_mean_gt0', 'OrderWeight_people_last_month_median_gt0', 
                     'OrderWeight_people_last_month_std_gt0',
                     'OrderWeight_people_mean_gt0', 'OrderWeight_people_median_gt0', 
                     'OrderWeight_people_std_gt0', 'OrderWeight_people_mean_inmonth', 
                     'OrderWeight_people_median_inmonth', 'OrderWeight_people_std_inmonth', 
                     'OrderWeight_people_cycle_mean', 'OrderWeight_people_cycle_median', 
                     'OrderWeight_people_cycle_std', 'OrderWeight_people_cycle_max', 
                     'OrderWeight_people_cycle_min', 'OrderWeight_people_cycle_mean_gt0', 
                     'OrderWeight_people_cycle_median_gt0', 'OrderWeight_people_cycle_std_gt0', 
                     'OrderWeight_mean', 'OrderWeight_median', 'OrderWeight_std', 'OrderWeight_max', 
                     'OrderWeight_min', 'OrderWeight_mean_gt0', 'OrderWeight_median_gt0',
                     'OrderWeight_people_last_month_week_mean',          
                     'OrderWeight_people_last_month_week_std',           
                     'OrderWeight_people_last_month_week_max',
                     'OrderWeight_people_last_month_week_min',           
                     'OrderWeight_people_last_month_week_mean_gt0',
                     "OrderWeight_people_last_month_week_median_gt0",
                     'OrderWeight_people_last_month_week_std_gt0',
                     'OrderWeight_std_gt0', 'is_songkarn', 'is_makabusha',
                     "day_of_week_sin", "day_of_week_cos"])

# Optional: preview result
print(test_df.head())

### Zero-inflated

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, log_loss
from sklego.meta import ZeroInflatedRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


def autotrain4(df, test_df): # Removed unused 'feature' parameter
    regions = df['MODEL_4'].unique()
    model_list = []
    all_predictions = []
    feature_importances_records = []
    classifier_list = []

    target = 'OrderWeight'
    
    base_features = ['CustomerBKey', 'ProductForPlan10', 'ProductForPlan1', 'ProductBKey',
       'ProductForPlan8', 'day_of_week',
       'month', 'week', 'OrderWeight_people_mean', 'OrderWeight_people_median',
       'OrderWeight_people_std', 'OrderWeight_people_max',
       'OrderWeight_people_min', 'OrderWeight_people_mean_gt0',
       'OrderWeight_people_median_gt0', 'OrderWeight_people_std_gt0',
       'OrderWeight_people_mean_inmonth', 'OrderWeight_people_median_inmonth',
       'OrderWeight_people_std_inmonth', 'OrderWeight_people_cycle_mean', 
       'OrderWeight_people_cycle_median', 'OrderWeight_people_cycle_std', 
       'OrderWeight_people_cycle_max', 'OrderWeight_people_cycle_min', 
       'OrderWeight_people_cycle_mean_gt0', 'OrderWeight_people_cycle_median_gt0', 
       'OrderWeight_people_cycle_std_gt0', 
       'OrderWeight_mean', 'OrderWeight_median', 'OrderWeight_std', 'OrderWeight_max',
       'OrderWeight_min', 'OrderWeight_mean_gt0', 'OrderWeight_median_gt0',
       'OrderWeight_std_gt0', 'is_new_year', 'is_songkarn', 'is_thudjene',
       'is_makabusha']
    
    categorical_features = [
    'CustomerBKey', 'ProductForPlan10', 'day_of_week', 'month', 'week', 
    'ProductForPlan1', 'ProductBKey', 'ProductForPlan8',
    'is_new_year', 'is_songkarn', 'is_thudjene', 'is_makabusha', 'CustomerBKey'
    ]

    # --- Step 1: Create the binary target 'has_sales' ---
    df['has_sales'] = (df[target] > 0).astype(int)
    # test_df does not have 'units_sold', so 'has_sales' cannot be created directly.
    # The classifier's prediction will provide this information for test_df.

    # --- Define a fixed random state for reproducibility ---
    FIXED_RANDOM_STATE = 42

    for region_pred in regions:
        print(f"\nProcessing Region {region_pred}...")

        # Filter training data for the current region
        region_mask = train_df['MODEL_4'] == region_pred
        region_df = train_df[region_mask].copy()

        # Filter test data for the current region
        test_region_mask = test_df['MODEL_4'] == region_pred
        region_test_df = test_df[test_region_mask].copy()

        # Skip if no training data for the current region
        if region_df.empty:
            print(f"No training data for region {region_pred}. Skipping.")
            continue

        # Define X and y for the current region's training data
        X_region = region_df[base_features]
        y_region = region_df[target]

        # Split region data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            X_region, y_region,
            test_size=0.2,
            random_state=FIXED_RANDOM_STATE
        )

        # Initialize LGBMClassifier for the zero-inflated part (predicting presence of sales)
        classifier = LGBMClassifier(
            objective='binary',
            metric='logloss',
            n_estimators=150,
            learning_rate=0.05,
            num_leaves=31,
            verbose=-1, # Suppress verbose output
            categorical_feature=categorical_features,
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1 # Use all available CPU cores
        )

        # Initialize LGBMRegressor for the regression part (predicting sales amount when present)
        # 'tweedie' objective is suitable for target variables with many zeros and positive values
        regressor = LGBMRegressor(
            objective='tweedie',
            tweedie_variance_power=1.5, # Power parameter for Tweedie distribution
            n_estimators=200,
            learning_rate=0.1,
            max_depth=6,
            categorical_feature=categorical_features,
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1,
            verbose=-1,
            boosting_type='goss'
        )

        # Combine classifier and regressor into a ZeroInflatedRegressor
        zir_model = ZeroInflatedRegressor(
            classifier=classifier,
            regressor=regressor
        )

        # Train the Zero Inflated Regressor model
        zir_model.fit(X_train, y_train)
        model_list.append((region_pred, zir_model))

        # --- Validation on the current region's validation set ---
        # Predict probability of having sales using the classifier
        prob_has_sales = zir_model.classifier_.predict_proba(X_val)[:, 1]
        # Predict sales amount using the regressor
        y_pred_reg = zir_model.regressor_.predict(X_val)
        # Combine predictions: if probability of sales > 0.6, use regressor's prediction, else 0
        y_pred_final = np.where(prob_has_sales > 0.35, y_pred_reg, 0)
        # Clip predictions to ensure non-negative values
        y_pred_final = np.clip(y_pred_final, a_min=0, a_max=None)

        # Calculate Mean Absolute Error (MAE) for validation
        mae_score = mean_absolute_error(y_val, y_pred_final)
        print(f"  Region {region_pred} MAE: {mae_score:.4f}")

        # --- Feature Importance ---
        # Get feature importances from the regressor model
        importances = zir_model.regressor_.feature_importances_
        for feature, importance in zip(base_features, importances):
            feature_importances_records.append({
                'region': region_pred,
                'feature': feature,
                'importance': importance
            })

        # --- Test Predictions for the current region ---
        if not region_test_df.empty:
            X_test = region_test_df[base_features]
            # Predict probability of sales for test data
            prob_test = zir_model.classifier_.predict_proba(X_test)[:, 1]
            # Predict sales amount for test data
            y_pred_test_reg = zir_model.regressor_.predict(X_test)
            # Combine test predictions
            y_pred_test_final = np.where(prob_test > 0.6, y_pred_test_reg, 0)
            # Clip test predictions
            y_pred_test_final = np.clip(y_pred_test_final, a_min=0, a_max=None)

            # Add predictions to the region's test DataFrame
            region_test_df['predicted_OrderWeight'] = y_pred_test_final
            # Append the 'id' and 'predicted_OrderWeight' columns to all_predictions list
            all_predictions.append(region_test_df[['id', 'predicted_OrderWeight']])

    # --- Combine all region predictions into a final test DataFrame ---
    if all_predictions:
        # Concatenate all regional predictions
        prediction_df = pd.concat(all_predictions, ignore_index=True)
        # Merge predictions back to the original test_df based on 'id'
        final_test_df = test_df.merge(
            prediction_df,
            on='id',
            how='left'
        )
    else:
        # If no predictions were made (e.g., all regions had no training data),
        # create a copy of the original test_df and initialize 'predicted_OrderWeight' to 0
        final_test_df = test_df.copy()
        final_test_df['predicted_OrderWeight'] = 0

    # Fill any remaining missing predictions (e.g., for regions not processed) with 0
    final_test_df['predicted_OrderWeight'] = final_test_df['predicted_OrderWeight'].fillna(0)

    return model_list, final_test_df, feature_importances_records

In [ ]:
feature_importances_records = []
model_list, test_df, feature_importances_records = autotrain4(train_df, test_df)

### CRAZY

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, log_loss
from xgboost import XGBRegressor, XGBClassifier
from xgboost.callback import EarlyStopping
from sklearn.metrics import roc_curve

def autotrain5(df, test_df):
    people = df['CustomerBKey'].unique()
    model_list = []
    all_predictions = []
    feature_importances_records = []

    target = 'OrderWeight'
    base_features = ['ProductForPlan10', 'ProductForPlan1', 'ProductBKey', 
                     'MODEL_4', 'ProductForPlan8', 'day_of_week', 
                     'month', 'week', 'OrderWeight_people_mean', 'OrderWeight_people_median', 
                     'OrderWeight_people_std', 'OrderWeight_people_max', 'OrderWeight_people_min', 
                     'OrderWeight_people_last_month_mean', 'OrderWeight_people_last_month_std', 
                     'OrderWeight_people_last_month_max', 'OrderWeight_people_last_month_min', 
                     'OrderWeight_people_last_month_mean_gt0', 'OrderWeight_people_last_month_median_gt0', 
                     'OrderWeight_people_last_month_std_gt0',
                     'OrderWeight_people_mean_gt0', 'OrderWeight_people_median_gt0', 
                     'OrderWeight_people_std_gt0', 'OrderWeight_people_mean_inmonth', 
                     'OrderWeight_people_median_inmonth', 'OrderWeight_people_std_inmonth', 
                     'OrderWeight_people_cycle_mean', 'OrderWeight_people_cycle_median', 
                     'OrderWeight_people_cycle_std', 'OrderWeight_people_cycle_max', 
                     'OrderWeight_people_cycle_min', 'OrderWeight_people_cycle_mean_gt0', 
                     'OrderWeight_people_cycle_median_gt0', 'OrderWeight_people_cycle_std_gt0', 
                     'OrderWeight_mean', 'OrderWeight_median', 'OrderWeight_std', 'OrderWeight_max', 
                     'OrderWeight_min', 'OrderWeight_mean_gt0', 'OrderWeight_median_gt0',
                     'OrderWeight_people_last_month_week_mean',          
                     'OrderWeight_people_last_month_week_std',           
                     'OrderWeight_people_last_month_week_max',
                     'OrderWeight_people_last_month_week_min',           
                     'OrderWeight_people_last_month_week_mean_gt0',
                     "OrderWeight_people_last_month_week_median_gt0",
                     'OrderWeight_people_last_month_week_std_gt0',
                     'OrderWeight_std_gt0', 'is_songkarn', 'is_makabusha',
                     "day_of_week_sin", "day_of_week_cos"]
    
    categorical_features = [
    'ProductForPlan1', 'ProductBKey', 'MODEL_4', 'ProductForPlan8',
    'is_songkarn', 'is_makabusha', 'ProductForPlan10',
    'day_of_week', 'month', 'week',
    ]


    # Create binary target
    df['has_sales'] = (df[target] > 0).astype(int)

    FIXED_RANDOM_STATE = 42

    for item in people:
        print(f"\nProcessing Item {item}...")
        item_df = df[df['CustomerBKey'] == item].copy()
        item_test_df = test_df[test_df['CustomerBKey'] == item].copy()

        current_features = base_features

        missing = [col for col in current_features if col not in item_df.columns]
        if missing:
            print(f"Item {item} missing features: {missing}")
            continue

        X_item = item_df[current_features]
        y_sales_item = item_df[target]
        y_has_sales_item = item_df['has_sales']

        # Train classifier
        print(f"  Training classifier for item {item}...")
        X_train_clf, X_val_clf, y_train_clf, y_val_clf = train_test_split(
            X_item, y_has_sales_item, test_size=0.2, stratify=y_has_sales_item,
            random_state=FIXED_RANDOM_STATE
        )

        classifier = XGBClassifier(
            objective='binary:logistic',
            n_estimators=200,
            learning_rate=0.05,
            max_depth=10,
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1,
            enable_categorical=True  # only available if using recent xgboost & categorical inputs encoded correctly
        )

        classifier.fit(
            X_train_clf,
            y_train_clf,
            eval_set=[(X_val_clf, y_val_clf)],
            verbose=False
        )

        val_probs = classifier.predict_proba(X_val_clf)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_val_clf, val_probs)
        optimal_threshold = thresholds[np.argmax(tpr - fpr)]

        print(f"  Item {item} Optimal Threshold: {optimal_threshold:.4f}")

        # Apply threshold to full item_df and test set
        item_df['prob_has_sales'] = classifier.predict_proba(X_item)[:, 1]
        item_test_df['prob_has_sales'] = classifier.predict_proba(item_test_df[current_features])[:, 1]


        clf_logloss = log_loss(y_val_clf, classifier.predict_proba(X_val_clf)[:, 1])
        print(f"  Item {item} Classifier LogLoss: {clf_logloss:.4f}")

        # Train regressor
        print(f"  Training regressor for item {item}...")
        regressor_features = current_features + ['prob_has_sales']

        X_reg = item_df[regressor_features]
        y_reg = item_df[target]

        X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(
            X_reg, y_reg, test_size=0.2, random_state=FIXED_RANDOM_STATE
        )

        regressor = XGBRegressor(
            objective='reg:tweedie',
            tweedie_variance_power=1.5,
            n_estimators=400,
            learning_rate=0.01,
            max_depth=10,
            random_state=FIXED_RANDOM_STATE,
            n_jobs=-1,
            eval_metric='mae',
            verbosity=0,
            enable_categorical=True
        )

        # Removed: early_stopping_callback_xgb definition
        regressor.fit(
            X_train_reg,
            y_train_reg,
            # Removed: eval_set and callbacks arguments from regressor.fit()
        )

        importances = regressor.feature_importances_
        for feature_name, importance_value in zip(regressor_features, importances):
            feature_importances_records.append({
                'item_id': item,
                'feature_name': feature_name,
                'importance_value': importance_value
            })

        importance_df = pd.DataFrame({
            'feature': regressor_features,
            'importance': importances
        }).sort_values(by='importance', ascending=False)

        top5 = importance_df.head(5)
        print(f"  Top 5 important features for item {item}:")
        print(top5.to_string(index=False))

        val_pred_reg = regressor.predict(X_val_reg)
        final_val_pred = np.where(
            X_val_reg['prob_has_sales'] > optimal_threshold,
            val_pred_reg,
            0
        )
        final_val_pred[final_val_pred < 0] = 0

        mae_score = mean_absolute_error(y_val_reg, final_val_pred)
        print(f"  Item {item} MAE: {mae_score:.4f}")

        # Predict on test set
        if item_test_df.empty:
            print(f"  No test data for item {item}.")
            continue

        X_test_combined = item_test_df[regressor_features]
        y_pred_test_reg = regressor.predict(X_test_combined)
        y_pred_test_final = np.where(
            X_test_combined['prob_has_sales'] > optimal_threshold,
            y_pred_test_reg,
            0
        )
        y_pred_test_final[y_pred_test_final < 0] = 0

        item_test_df['predicted_sales'] = y_pred_test_final
        all_predictions.append(item_test_df[['id', 'predicted_sales']])

    # Combine predictions
    if all_predictions:
        prediction_df = pd.concat(all_predictions, ignore_index=True)
        test_df = test_df.merge(prediction_df, on='id', how='left')
        test_df['predicted_sales'] = test_df['predicted_sales'].fillna(0)

    return model_list, test_df, feature_importances_records


In [ ]:
feature_importances_records = []
model_list, test_df, feature_importances_records = autotrain5(train_df, test_df)

# Predict

In [ ]:
# Replace OrderWeight with predicted_sales
test_df['OrderWeight'] = test_df['predicted_sales']

# Drop day_of_week and month
test_df = test_df.drop(columns=['day_of_week', 'month', 'predicted_sales', 'week', 'ProductForPlan8',
                                'OrderWeight_people_mean', 'OrderWeight_people_median', 
                     'OrderWeight_people_std', 'OrderWeight_people_max', 'OrderWeight_people_min', 
                     'OrderWeight_people_last_month_mean', 'OrderWeight_people_last_month_std', 
                     'OrderWeight_people_last_month_max', 'OrderWeight_people_last_month_min', 
                     'OrderWeight_people_last_month_mean_gt0', 'OrderWeight_people_last_month_median_gt0', 
                     'OrderWeight_people_last_month_std_gt0',
                     'OrderWeight_people_mean_gt0', 'OrderWeight_people_median_gt0', 
                     'OrderWeight_people_std_gt0', 'OrderWeight_people_mean_inmonth', 
                     'OrderWeight_people_median_inmonth', 'OrderWeight_people_std_inmonth', 
                     'OrderWeight_people_cycle_mean', 'OrderWeight_people_cycle_median', 
                     'OrderWeight_people_cycle_std', 'OrderWeight_people_cycle_max', 
                     'OrderWeight_people_cycle_min', 'OrderWeight_people_cycle_mean_gt0', 
                     'OrderWeight_people_cycle_median_gt0', 'OrderWeight_people_cycle_std_gt0', 
                     'OrderWeight_mean', 'OrderWeight_median', 'OrderWeight_std', 'OrderWeight_max', 
                     'OrderWeight_min', 'OrderWeight_mean_gt0', 'OrderWeight_median_gt0',
                     'OrderWeight_people_last_month_week_mean',          
                     'OrderWeight_people_last_month_week_std',           
                     'OrderWeight_people_last_month_week_max',
                     'OrderWeight_people_last_month_week_min',           
                     'OrderWeight_people_last_month_week_mean_gt0',
                     "OrderWeight_people_last_month_week_median_gt0",
                     'OrderWeight_people_last_month_week_std_gt0',
                     'OrderWeight_std_gt0', 'is_songkarn', 'is_makabusha',
                     "day_of_week_sin", "day_of_week_cos"])

# Optional: preview result
print(test_df.head())


In [ ]:
test_df

In [ ]:
import random

unique_test_pairs = test_df[['CustomerBKey', 'ProductBKey']].drop_duplicates().values.tolist()

if unique_test_pairs:
    random_pair = random.choice(unique_test_pairs)
    random_store_id, random_item_id = random_pair

    print(f"\nRandomly selected pair: CustomerID = {random_store_id}, Item ID = {random_item_id}")

    pair_df = test_df[
        (test_df['CustomerBKey'] == random_store_id) &
        (test_df['ProductBKey'] == random_item_id)
    ].copy()

    item = pair_df["ProductForPlan10"].iloc[0]
    pair_df_21_days = pair_df.head(21)

    # --- Plot using Plotly ---
    fig = px.line(
        pair_df_21_days,
        x='OrderDate',
        y='OrderWeight',
        title=f'Store ID: {random_store_id}, Item ID: {random_item_id} - Predicted Sales (21 Days), item: {item}',
        labels={'OrderDate': 'Date', 'OrderWeight': 'Predicted Sales'}
    )

    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Predicted Sales',
        xaxis_tickangle=-45,
        template='plotly_white'
    )

    fig.show()

else:
    print("\nNo unique store-item pairs found in the test data to plot.")

In [ ]:
train_df[(train_df['ProductBKey'] == random_item_id) & (train_df['CustomerBKey'] == random_store_id)].tail(30)


In [ ]:
train_df[(train_df['CustomerBKey'] == random_item_id)].head(30)


In [ ]:
zero_count = (test_df['OrderWeight'] == 0).sum()
print(f"Number of predicted_sales == 0: {zero_count}")

In [ ]:
def zero_15_of_every_21_rows(df, target_col='OrderWeight'):
    df1 = df.copy()
    total_rows = len(df1)

    for start in range(0, total_rows, 21):
        end = min(start + 21, total_rows)
        block_indices = list(range(start, end))
        if len(block_indices) < 12:
            continue  # skip small leftover blocks

        # Randomly pick 15 indices in the block to zero out
        zero_indices = np.random.choice(block_indices, size=12, replace=False)
        df1.loc[zero_indices, target_col] = 0

    return df1
test_df_zero = zero_15_of_every_21_rows(test_df)

In [ ]:
test_df.to_parquet("Submission.parquet", index=False)

In [ ]:
test_df_zero.to_parquet("Submissionzero.parquet", index=False)

In [ ]:
test_df['OrderWeight'] *= 0.5

In [ ]:
import random

unique_test_pairs = test_df[['CustomerBKey', 'ProductBKey']].drop_duplicates().values.tolist()

if unique_test_pairs:
    random_pair = random.choice(unique_test_pairs)
    random_store_id, random_item_id = random_pair

    print(f"\nRandomly selected pair: CustomerID = {random_store_id}, Item ID = {random_item_id}")

    pair_df = test_df_zero[
        (test_df_zero['CustomerBKey'] == random_store_id) &
        (test_df_zero['ProductBKey'] == random_item_id)
    ].copy()

    item = pair_df["ProductForPlan10"].iloc[0]
    pair_df_21_days = pair_df.head(21)

    # --- Plot using Plotly ---
    fig = px.line(
        pair_df_21_days,
        x='OrderDate',
        y='OrderWeight',
        title=f'Store ID: {random_store_id}, Item ID: {random_item_id} - Predicted Sales (21 Days), item: {item}',
        labels={'OrderDate': 'Date', 'OrderWeight': 'Predicted Sales'}
    )

    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Predicted Sales',
        xaxis_tickangle=-45,
        template='plotly_white'
    )

    fig.show()

else:
    print("\nNo unique store-item pairs found in the test data to plot.")

In [ ]:
train_df[train_df['ProductBKey'] == random_item_id].head(30)

In [ ]:
zero_count = (test_df_zero['OrderWeight'] == 0).sum()
print(f"Number of predicted_sales == 0: {zero_count}")